In [5]:
from time import time
import lightgbm as lgb
import numpy as np
import sys
# sys.path.append('..\Python Scripts\pipeline')
import warnings
warnings.filterwarnings('ignore') 
import seaborn as sns
from sklearn import preprocessing
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.model_selection import KFold, TimeSeriesSplit
from scipy.stats import uniform
import datetime
from sklearn.ensemble import RandomForestClassifier
import lightgbm
from scipy import stats
# from hyperopt import hp, tpe
# from hyperopt.fmin import fmin
from sklearn.model_selection import cross_val_score, StratifiedKFold
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV, validation_curve, KFold
from sklearn.metrics import roc_auc_score
import seaborn as sns
%matplotlib inline
import gc

In [4]:
def learn_lgbm(epochs, X, y,  params):
        
    '''
    Функция делает кросс-валидацию по 5 фолдам с кучей эпох, каждый раз обучая новыый классификатор. 
    Возвращает кучу всего, да еще и feature importance считает
    '''
        
    feature_importance_df = pd.DataFrame()
    list_rocauc_test = []
    list_rocauc_train = []

    for epoch in range(epochs):
        
        skf = StratifiedKFold(n_splits=5, shuffle=True)
        oof_preds = np.zeros(X.loc[:, X.columns].shape[0])

        for n_fold, (trn_idx, tst_idx) in enumerate(skf.split(X, y)):

            trn_x, trn_y = X.loc[:, X.columns].iloc[trn_idx], y.iloc[trn_idx]
            tst_x, tst_y = X.loc[:, X.columns].iloc[tst_idx], y.iloc[tst_idx]

            clf = lightgbm.LGBMClassifier(**params)

            clf.fit(trn_x, trn_y, 
                    eval_set= [(tst_x, tst_y)], 
                    eval_metric='auc', verbose=False, early_stopping_rounds=100  #30
                   )

            oof_preds[tst_idx] = clf.predict_proba(tst_x, num_iteration=clf.best_iteration_)[:, 1]
            pred_train = clf.predict_proba(trn_x, num_iteration=clf.best_iteration_)[:, 1]


            fold_importance_df = pd.DataFrame()
            fold_importance_df["feature"] = X.loc[:, X.columns].columns
            fold_importance_df["importance"] = clf.feature_importances_
            fold_importance_df["fold"] = n_fold + 1
            feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

            rocauc_test = roc_auc_score(tst_y, oof_preds[tst_idx]) 
            rocauc_train = roc_auc_score(trn_y, pred_train) 

            list_rocauc_test.append(rocauc_test)
            list_rocauc_train.append(rocauc_train)
            print(rocauc_train, rocauc_test)
            
        return clf, feature_importance_df, list_rocauc_train , list_rocauc_test 


In [10]:
df.replace(-99999, np.nan,  inplace=True)

X = df.drop(['target'], axis=1)
y = df.target

AttributeError: drop

# Random forest

In [20]:
clf_rf = RandomForestClassifier(min_samples_leaf=300, n_estimators=500, max_depth=4, max_leaf_nodes=10,
                                random_state=1)
clf_rf.fit(X, y)   
print('useful variables:',(clf_rf.feature_importances_>0).sum())

list_to_stay = list(X.columns[clf_rf.feature_importances_>0])
len(list_to_stay)

useful variables: 0


0

# Gradient Boosting

In [24]:
params = {
    'n_estimators': 200,
    'learning_rate': 0.1,
    'num_leaves': 20,
    'max_depth': 1,
    'min_child_weight': 10,
    'lambda_l1':2,
    'lambda_l2':3,
    'min_data_in_leaf' :10,
    'min_sum_hessian_in_leaf' : 0.0001,
    'bagging_fraction' : 0.8,
    'max_bin': 12,
    'feature_fraction' : 0.9,
    'bagging_freq' : 100,
    'min_gain_to_split': 0.1 }


clf, feature_importance, list_gini_test,  list_gini_train = learn_lgbm(1, X, y, params)
fimp = feature_importance[feature_importance['importance']>0].groupby('feature').agg('mean').drop('fold', axis=1).sort_values(by='importance', ascending=False).reset_index()

LightGBMError: Multiclass objective and metrics don't match

# SHAP

In [ ]:
get_ipython().run_line_magic('matplotlib', 'inline')
import shap

shap.initjs()
explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X)

shap_sum = np.abs(shap_values).mean(axis=0)

importance_df = pd.DataFrame([X.columns.tolist(), shap_sum.tolist()]).T
importance_df.columns = ['column_name', 'shap_importance']
importance_df = importance_df.sort_values('shap_importance', ascending=False)
importance_df = importance_df.reset_index(drop=True)
sorted_shap_importance = importance_df[importance_df['shap_importance'] > 0]

shapp = sorted_shap_importance['column_name']


In [ ]:
feats_union = list(set(list_to_stay) | set(shapp) | set(fimp['feature']))

# Individual Gini

In [ ]:
def individual_gini(i, df, feature):
    print(i, ' feature: ', feature)
    df = df.fillna(-9999999)
    
    if df[feature].dtypes == 'O':
        x = pd.get_dummies(df[feature]).values
        obvious_gini = 0
    else:
#         print('\t  Counting obvious gini...')
        x = df[feature].values.reshape(-1, 1)    
        obvious_gini = round(abs(roc_auc_score(df.target, x) * 2 - 1), 3)

#     print('\t  Now count gini using decision tree')    
    parameters = {'min_weight_fraction_leaf': [0.01, 0.025, 0.05, 0.1]}
    param_dist = {'min_weight_fraction_leaf': np.random.uniform(0.001, 0.1, 4) }
    dt = DecisionTreeClassifier(random_state=1)
        
    cv = StratifiedKFold(3, random_state=1)
    clf = GridSearchCV(dt, parameters,cv=cv,scoring='roc_auc', n_jobs=30)
    print('\t  Run grid search...')
#     clf = RandomizedSearchCV(dt, param_distributions=param_dist, cv=cv, verbose=0, n_jobs=10, scoring='roc_auc')
    clf.fit(x, df.target)
    
    true_gini = round(abs(clf.best_score_ * 2 - 1), 3)
    print('\t  True gini is ', true_gini)
    print('\t  Clf is fitted!\n-------------------------------------------')
    
    return max(obvious_gini, true_gini), obvious_gini, true_gini, obvious_gini > true_gini



df_gini = pd.DataFrame(index = df.columns.values, columns = ['max_gini', 'obvious', 'true_gini', 'obvious_is_max'])

for i, col in enumerate(df.columns):
    gini = individual_gini(i, df, col)
    df_gini.loc[[col], ['max_gini', 'obvious', 'true_gini', 'obvious_is_max']] = gini

df_gini_all = pd.DataFrame(index =  df.columns.values, columns = ['max_gini', 'obvious', 'true_gini', 'obvious_is_max'])
gini_to_drop = list(df_gini.loc[df_gini.max_gini < 0.01].index)

# Forward

In [2]:
def forward(vars_list, all_vars, X_train, X_test, y_train, y_test, params):
    res=[]
    q=0
    
    for col in list(set(all_vars) - set(vars_list)):
        q+=1
        vars_stay = vars_list + [col]
        
        lgb = lightgbm.LGBMClassifier(**params)
        lgb.fit(X_train[vars_stay], y_train, eval_set=[(X_test[vars_stay], y_test)], 
                        eval_metric='auc', verbose=False, early_stopping_rounds=100,) 
        
        tr = roc_auc_score(y_train, lgb.predict_proba(X_train[vars_stay])[:,1])
        te = roc_auc_score(y_test, lgb.predict_proba(X_test[vars_stay])[:,1])
        
        res.append([col, tr, te])
        print(q, ' feature: ', col,  '\ttrain: ', round(tr, 3),  '\ttest: ', round(te, 3))
                    
    forw = pd.DataFrame(res, columns = ['feature', 'train', 'test'])
    forw['train_mean'] = forw['train']
    return forw.sort_values(by='train_mean', ascending=False)[['feature', 'train', 'test']].reset_index(drop=True).loc[:0, :]


columns_forward = list(df.columns.drop(['target']))

list_to_add = columns_forward.copy()
list_start = []
temp_df_score = pd.DataFrame(columns=['feature', 'train', 'test'])


X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.3, random_state=0)


while len(list_to_add) != 0:
    forward_iteration = forward(list_start, columns_forward, X_train, X_test, y_train, y_test, params)

    temp_df_score = temp_df_score.append(forward_iteration, ignore_index=True)
    print(temp_df_score)
    var = forward_iteration.values[0][0]
    list_start.append(var)
    list_to_add.remove(var)
    print(var)


NameError: name 'df' is not defined

# Backward

In [ ]:
def backward(vars_list, X_train, X_test, y_train, y_test,  best_params):
    res=[]
    q=0

    for item in vars_list:
    
        vars_stay = list(set(vars_list) - set([item]))
        q+=1
#         print('\n', vars_stay)
        lgb = lightgbm.LGBMClassifier(**params)
        lgb.fit(X_train[vars_stay], y_train, eval_metric='auc', verbose=False)
        
        tr =  roc_auc_score(y_train, lgb.predict_proba(X_train[vars_stay])[:,1]) 
        te = roc_auc_score(y_test, lgb.predict_proba(X_test[vars_stay])[:,1]) 
        res.append([item, tr, te])
        
        print(q, ' feature: ', item, '\ttrain: ', round(tr, 3), '\ttest: ', round(te, 3))
    backw = pd.DataFrame(res, columns = ['var', 'train', 'test'])
    
    return backw.sort_values(by='train', ascending=False)[['var', 'train', 'test']].reset_index(drop=True).loc[:0, :]


X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y_final, test_size = 0.3, random_state = 0)

list_start = list(var_list_final)

bw = pd.DataFrame(res, columns = ['var', 'train', 'test']).sort_values(by='train', ascending=False)[['var', 'train', 'test']].reset_index(drop=True).loc[:0, :]
temp_df_new = temp_df_new.append(bw, ignore_index=True)

for i in range(len(var_list_final)):


    bw = backward(list_start, X_train_final[var_list_final], X_test_final[var_list_final], y_train_final, y_test_final, best_params)
    temp_df_new = temp_df_new.append(bw, ignore_index=True)
    print(temp_df_new)
    var = bw.values[0][0]
    
    list_start.remove(var)